<div align="center">

  <a href="https://ultralytics.com/yolo" target="_blank">
    <img width="1024", src="https://raw.githubusercontent.com/ultralytics/assets/main/yolov8/banner-yolov8.png"></a>

  [中文](https://docs.ultralytics.com/zh/) | [한국어](https://docs.ultralytics.com/ko/) | [日本語](https://docs.ultralytics.com/ja/) | [Русский](https://docs.ultralytics.com/ru/) | [Deutsch](https://docs.ultralytics.com/de/) | [Français](https://docs.ultralytics.com/fr/) | [Español](https://docs.ultralytics.com/es/) | [Português](https://docs.ultralytics.com/pt/) | [Türkçe](https://docs.ultralytics.com/tr/) | [Tiếng Việt](https://docs.ultralytics.com/vi/) | [العربية](https://docs.ultralytics.com/ar/)

  <a href="https://github.com/ultralytics/ultralytics/actions/workflows/ci.yml"><img src="https://github.com/ultralytics/ultralytics/actions/workflows/ci.yml/badge.svg" alt="Ultralytics CI"></a>
  <a href="https://colab.research.google.com/github/ultralytics/notebooks/blob/main/notebooks/how-to-count-the-objects-using-ultralytics-yolo.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
  
  <a href="https://ultralytics.com/discord"><img alt="Discord" src="https://img.shields.io/discord/1089800235347353640?logo=discord&logoColor=white&label=Discord&color=blue"></a>
  <a href="https://community.ultralytics.com"><img alt="Ultralytics Forums" src="https://img.shields.io/discourse/users?server=https%3A%2F%2Fcommunity.ultralytics.com&logo=discourse&label=Forums&color=blue"></a>
  <a href="https://reddit.com/r/ultralytics"><img alt="Ultralytics Reddit" src="https://img.shields.io/reddit/subreddit-subscribers/ultralytics?style=flat&logo=reddit&logoColor=white&label=Reddit&color=blue"></a>
  
  Welcome to the Object counting using Ultralytics YOLO11 🚀 notebook! <a href="https://github.com/ultralytics/ultralytics">YOLO11</a> is the latest version of the YOLO (You Only Look Once) AI models developed by <a href="https://ultralytics.com">Ultralytics</a>. We hope that the resources in this notebook will help you get the most out of YOLO11. Please browse the YOLO11 <a href="https://docs.ultralytics.com/">Docs</a> for details, raise an issue on <a href="https://github.com/ultralytics/ultralytics">GitHub</a> for support, and join our <a href="https://ultralytics.com/discord">Discord</a> community for questions and discussions!</div>

# Object Counting using Ultralytics YOLO11

This notebook serves as a starting point for [counting objects](https://docs.ultralytics.com/guides/object-counting/) in videos or live streams using the YOLO11 model.

### What is Object Counting?

- Object counting with YOLO11 involves accurate identification and counting of specific objects in videos and camera streams. YOLO11 excels in real-time applications, providing efficient and precise object counting for various scenarios like crowd analysis and surveillance, thanks to its state-of-the-art algorithms and deep learning capabilities.

### Advantages of Object Counting?

- **Resource Optimization**: Object counting facilitates efficient resource management by providing accurate counts, and optimizing resource allocation in applications like inventory management.
- **Enhanced Security**: Object counting enhances security and surveillance by accurately tracking and counting entities, aiding in proactive threat detection.
- **Informed Decision-Making**: Object counting offers valuable insights for decision-making, optimizing processes in retail, traffic management, and various other domains.

### Setup

pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://www.pepy.tech/projects/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [1]:
!pip install --upgrade ultralytics
!pip install --upgrade torch torchvision torchaudio
!pip show ultralytics


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Name: ultralytics
Version: 8.3.173
Summary: Ultralytics YOLO 🚀 for SOTA object detection, multi-object tracking, instance segmentation, pose estimation and image classification.
Home-page: https://ultralytics.com
Author: 
Author-email: Glenn Jocher <glenn.jocher@ultralytics.com>, Jing Qiu <jing.qiu@ultralytics.com>
License: AGPL-3.0
Location: /Users/qbit-glitch/Desktop/coding-projects/computer_vision_projects/ven2_cv/lib/python3.13/site-packages
Requires: matplotlib, numpy, opencv-python, pandas, pillow, psutil, py-cpuinfo, pyyaml, requests, scipy, torch, torchvision, tqdm, ultralytics-thop
Required-by: 


In [2]:
import ultralytics
import cv2
from ultralytics.utils.downloads import safe_download
from ultralytics import solutions

ultralytics.checks()

Ultralytics 8.3.173 🚀 Python-3.13.5 torch-2.7.1 CPU (Apple M4 Pro)
Setup complete ✅ (14 CPUs, 48.0 GB RAM, 688.4/926.4 GB disk)


#### Some additional packages required for mac

### Read the Video File

- You can either read the video file directly or stream the content from an RTSP (Real-Time Streaming Protocol) source, allowing for flexible video input depending on your needs.
- We will also set up the video writer to handle the output video writing.

In [3]:
# safe_download("https://github.com/ultralytics/notebooks/releases/download/v0.0.0/solutions-ci-demo.mp4")
cap = cv2.VideoCapture("cars-traffic.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH,
                                       cv2.CAP_PROP_FRAME_HEIGHT,
                                       cv2.CAP_PROP_FPS))

# Video writer
video_writer = cv2.VideoWriter("counting.avi",
                               cv2.VideoWriter_fourcc(*"mp4v"),
                               fps, (w, h))

### Define Region Coordinates

Here, we set the coordinates for specific regions to ensure accurate object tracking and analysis within the video or stream. This helps monitor and count objects effectively in different areas.

In [4]:
# Define region points
# region_points = [(20, 400), (1080, 400)]  # For line counting
region_points = [(20, 400), (1080, 400), (1080, 360), (20, 360)]  # For rectangle region counting
# region_points = [(20, 400), (1080, 400), (1080, 360), (20, 360), (20, 400)]  # For polygon region counting

### Initialize the ObjectCounter Class

- Now, let's initialize the `ObjectCounter` class to track and count objects in each frame of the video.

In [5]:
from ultralytics import YOLO

model = YOLO('yolo11s.pt')
model.to('mps')

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_runnin

In [6]:
# Init ObjectCounter
counter = solutions.ObjectCounter(
    show=True,  # Display the output
    region=region_points,  # Pass region points
    model=model,  # model="yolo11n-obb.pt" for object counting using YOLO11 OBB model.
    classes=[0, 2],  # If you want to count specific classes i.e person and car with COCO pretrained model.
    show_in=True,  # Display in counts
    # show_out=True,  # Display out counts
    line_width=2,  # Adjust the line width for bounding boxes and text display
)

Ultralytics Solutions: ✅ {'source': None, 'model': YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, 

### Process Video Frames

In this step, we will process each frame of the video to detect and analyze objects. This allows for real-time tracking and counting, based on the visual data in the frames.

In [ ]:
# Make sure codec is macOS-friendly
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS) or 30
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

video_writer = cv2.VideoWriter(
    'output.mp4', fourcc, fps, (frame_width, frame_height)
)

# Process video
while True:
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    results = counter(im0)  # count the objects
    frame = results.plot_im  # Make sure this matches (width, height)
    video_writer.write(results.plot_im)   # write the video frames

    # Quit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Quitting by user request.")
        break


cap.release()   # Release the capture
video_writer.release()
cv2.destroyAllWindows()
print("All resources released.")

0: 720x1280 34.3ms
Speed: 692.7ms track, 34.3ms solution per image at shape (1, 3, 720, 1280)

1: 720x1280 15.5ms
Speed: 46.2ms track, 15.5ms solution per image at shape (1, 3, 720, 1280)

2: 720x1280 14.7ms
Speed: 43.8ms track, 14.7ms solution per image at shape (1, 3, 720, 1280)

3: 720x1280 16.5ms
Speed: 44.9ms track, 16.5ms solution per image at shape (1, 3, 720, 1280)

4: 720x1280 14.4ms
Speed: 43.8ms track, 14.4ms solution per image at shape (1, 3, 720, 1280)

5: 720x1280 14.3ms
Speed: 44.7ms track, 14.3ms solution per image at shape (1, 3, 720, 1280)

6: 720x1280 14.0ms
Speed: 44.5ms track, 14.0ms solution per image at shape (1, 3, 720, 1280)

7: 720x1280 15.6ms
Speed: 45.4ms track, 15.6ms solution per image at shape (1, 3, 720, 1280)

8: 720x1280 14.2ms
Speed: 45.5ms track, 14.2ms solution per image at shape (1, 3, 720, 1280)

9: 720x1280 14.4ms
Speed: 45.2ms track, 14.4ms solution per image at shape (1, 3, 720, 1280)

10: 720x1280 16.0ms
Speed: 43.7ms track, 16.0ms solution pe

: 

![Fish Counting in the Sea Using Ultralytics YOLO11](https://github.com/ultralytics/docs/releases/download/0/conveyor-belt-packets-counting.avif)

Crafted with 💙 by [Ultralytics](https://ultralytics.com/)  

🌟 Explore and star the [Ultralytics Notebooks](https://github.com/ultralytics/notebooks/) to supercharge your AI journey! 🚀